In [68]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os


In [69]:
max_steps = 1000  # 最大迭代次数
learning_rate = 0.001  # 学习率
dropout = 0.9  # dropout时随机保留神经元的比例
data_dir = './lab9-tb/MNIST_DATA'  # 样本数据存储的路径
log_dir = './lab9-tb/MNIST_LOG'  # 输出日志保存的路径

# tf.compat.v1.disable_eager_execution()


In [70]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.33)
config.gpu_options.allow_growth = True


In [71]:
# 获取数据集，并采用采用one_hot编码
mnist = input_data.read_data_sets(data_dir, one_hot=True)


Extracting ./lab9-tb/MNIST_DATA\train-images-idx3-ubyte.gz
Extracting ./lab9-tb/MNIST_DATA\train-labels-idx1-ubyte.gz
Extracting ./lab9-tb/MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting ./lab9-tb/MNIST_DATA\t10k-labels-idx1-ubyte.gz


In [72]:
sess = tf.compat.v1.InteractiveSession(config=config)

In [73]:
with tf.name_scope('input'):
    x = tf.compat.v1.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.compat.v1.placeholder(tf.float32, [None, 10], name='y-input')


In [74]:
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)


In [75]:
# 初始化权重参数
def weight_variable(shape):
    initial = tf.compat.v1.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


In [76]:
# 初始化偏执参数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [77]:
# 绘制参数变化
def variable_summaries(var):
    with tf.name_scope('summaries'):
        # 计算参数的均值，并使用tf.summary.scaler记录
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)

        # 计算参数的标准差
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        # 使用tf.summary.scaler记录记录下标准差，最大值，最小值
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        # 用直方图记录参数的分布
        tf.summary.histogram('histogram', var)


In [82]:
# 构建神经网络
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # 设置命名空间
    with tf.name_scope(layer_name):
        # 调用之前的方法初始化权重w，并且调用参数信息的记录方法，记录w的信息
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        # 调用之前的方法初始化权重b，并且调用参数信息的记录方法，记录b的信息
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        # 执行wx+b的线性计算，并且用直方图记录下来
        with tf.name_scope('linear_compute'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('linear', preactivate)
        # 将线性输出经过激励函数，并将输出也用直方图记录下来
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
    # 返回激励层的最终输出
    return activations

In [83]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

In [84]:
# 创建dropout层
with tf.name_scope('dropout'):
    keep_prob = tf.compat.v1.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

ValueError: Expected scalar shape for dropout_4/Placeholder:0, saw shape: <unknown>.

In [85]:
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)


NameError: name 'dropped' is not defined

In [81]:
# 创建损失函数
with tf.name_scope('loss'):
    # 计算交叉熵损失（每个样本都会有一个损失）
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
        # 计算所有样本交叉熵损失的均值
        cross_entropy = tf.reduce_mean(diff)
    tf.summary.scalar('loss', cross_entropy)


NameError: name 'y' is not defined